In [76]:
import pandas as pd
import numpy as np

In [77]:
movies = pd.read_csv('C:/Users/ibn abbas/Desktop/New folder/recommender/movies_recommendation_system/movies.csv')


In [78]:
import re

In [79]:
def clean_title(title):
    return re.sub("[^a-zA-Z0-9 ]"," ", title)
    

In [80]:
movies["clean_title"] = movies["title"].apply(clean_title)

In [81]:
movies

,movieId,title,genres,clean_title
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story 1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji 1995
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men 1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Waiting to Exhale 1995
4,5,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II 1995
...,...,...,...,...
62418,209157,We (2018),Drama,We 2018
62419,209159,Window of the Soul (2001),Documentary,Window of the Soul 2001
62420,209163,Bad Poems (2018),Comedy|Drama,Bad Poems 2018
62421,209169,A Girl Thing (2001),(no genres listed),A Girl Thing 2001


In [82]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [83]:
vectorizer = TfidfVectorizer(ngram_range = (1,2))

In [84]:
# turnig the set of title into a matrix
tfidf = vectorizer.fit_transform(movies["clean_title"])

In [85]:
tfidf

<62423x168482 sparse matrix of type '<class 'numpy.float64'>'
	with 448766 stored elements in Compressed Sparse Row format>

creating the search function

In [86]:
from sklearn.metrics.pairwise import cosine_similarity 
import numpy as np
def search(title):
    title = clean_title(title)
    #changing the title set to vector matrix
    query_vec = vectorizer.transform([title])
    #finding the similarity to comapare the query of term from each of the title
    similarity = cosine_similarity(query_vec,tfidf).flatten()
    # find the title with the highest similarity
    indices = np.argpartition(similarity,-5)[-10:]
    #index the title
    result = movies.iloc[indices][::1]
    return result

building an interactive search engine using Jupyter

In [87]:
import ipywidgets as widgets
from IPython.display import display
#creating the input widget
movie_input = widgets.Text(
    value = "Toy story",
    description = "movie Title",
    disabled = False
)
movie_output_list = widgets.Output()
movie_input
#call this function when anything is type
def on_type(data):
    with movie_output_list:
        movie_output_list.clear_output()
        title= data["new"]
        if len(title) > 5:
            display(search(title))
# run on_type function if there is any change in the i nput
#hooking the input function with on_type
movie_input.observe(on_type, names="value")
display(movie_input,  movie_output_list)


Text(value='Toy story', description='movie Title')

Output()

Creating the recommendation system

In [88]:
rating = pd.read_csv("C:/Users/ibn abbas/Desktop/New folder/recommender/movies_recommendation_system/ratings.csv")

In [128]:
rating.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [129]:
rating
rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000095 entries, 0 to 25000094
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 762.9 MB


In [91]:
#Finding all the user that like the movies we like

In [132]:
movie_Id = 1

In [133]:
same_movies = rating[(rating["movieId" == movieId]) & (rating["rating"] > 4)]["userId"].unique()

KeyError: False

In [127]:
same_movies

array([     1,      2,      3, ..., 162539, 162540, 162541], dtype=int64)

Other moves like by the users that like the same movies we like

In [112]:
same_movies_recs = rating[(rating["userId"].isin(same_movies)) & (rating["rating"] > 4)]["movieId"]

In [113]:
same_movies_recs.value_counts()/len(same_movies)

318       0.326941
296       0.272484
2571      0.236205
356       0.225398
593       0.218661
            ...   
66539     0.000006
66720     0.000006
66773     0.000006
147322    0.000006
148685    0.000006
Name: movieId, Length: 32594, dtype: float64

In [114]:
# getting the movies rated by only 10% of users that like the same movies as us
same_movies_recs = same_movies_recs.value_counts()/len(same_movies)
same_movies_recs= same_movies_recs[same_movies_recs > .1]

In [115]:
same_movies_recs

318      0.326941
296      0.272484
2571     0.236205
356      0.225398
593      0.218661
260      0.212713
2959     0.209765
527      0.208303
858      0.201797
50       0.192433
4993     0.181498
1196     0.181337
7153     0.167037
2858     0.161416
5952     0.160621
1198     0.158929
110      0.153532
58559    0.141770
47       0.139835
1210     0.138687
608      0.138053
589      0.130431
4226     0.127464
79132    0.126880
2028     0.126714
1221     0.123060
1        0.120727
1136     0.118375
1197     0.115497
1193     0.114984
2762     0.114023
541      0.111311
3578     0.110741
1270     0.110689
4973     0.108799
1704     0.104004
7361     0.100895
Name: movieId, dtype: float64

finding how much movies the users like in our dataset set 
finding all the users that actually the movies recommened to a user like

In [116]:
#find all users that like the movie a user like
all_users = rating[(rating["movieId"].isin(same_movies_recs)) & (rating["rating"] > 4)]

In [117]:
all_users

,userId,movieId,rating,timestamp


In [100]:
#find the percentage of all users that recommended this movies
all_users_record =  all_users["movieId"].value_counts()/len(all_users["userId"].unique())

In [101]:
#all_users_record
all_users["movieId"]


5101        1
9939        1
11842       1
12232       1
12504       1
           ..
24996419    1
24997459    1
24997758    1
24998300    1
24998525    1
Name: movieId, Length: 18835, dtype: int64

In [102]:
all_users["userId"]

5101            36
9939            75
11842           86
12232           90
12504           93
             ...  
24996419    162519
24997459    162524
24997758    162527
24998300    162530
24998525    162533
Name: userId, Length: 18835, dtype: int64

 Creating a recommendation score

In [103]:
record_percentage = pd.concat([same_movies_recs,all_users_record], axis=1)
record_percentage.columns =["same","all"]

In [104]:
record_percentage

,same,all
1,1.000000,1.0
318,0.445607,NaN
260,0.403770,NaN
356,0.370215,NaN
296,0.367295,NaN
...,...,...
953,0.103053,NaN
551,0.101195,NaN
1222,0.100876,NaN
745,0.100345,NaN


In [105]:
 all_users["movieId"].value_counts()

1    18835
Name: movieId, dtype: int64

In [106]:
all_users_record =  all_users["movieId"].value_counts()

In [107]:
all_users_record

1    18835
Name: movieId, dtype: int64

In [108]:
record_percentage["score"] = record_percentage["same"]/record_percentage["all"]

In [109]:
record_percentage

,same,all,score
1,1.000000,1.0,1.0
318,0.445607,NaN,NaN
260,0.403770,NaN,NaN
356,0.370215,NaN,NaN
296,0.367295,NaN,NaN
...,...,...,...
953,0.103053,NaN,NaN
551,0.101195,NaN,NaN
1222,0.100876,NaN,NaN
745,0.100345,NaN,NaN
